<a href="https://colab.research.google.com/github/belerico/aml-project/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [6]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [0]:
os.chdir('/content/drive/My Drive/AML')

# REPO CLONE
If you want to clone the repo[link text](https://)

In [18]:
# Clone the entire repo in current directory
!git clone -l -s https://github.com/belerico/aml-project.git aml-project
%cd aml-project
!ls

Cloning into 'aml-project'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 49 (delta 18), reused 40 (delta 13), pack-reused 0
Unpacking objects: 100% (49/49), done.
/content/drive/My Drive/AML/aml-project
dataset  LICENSE  main.py  models.py  README.md  references  test  utils.py


# Execution


In [0]:
from dataset import get_data
from models import get_deep_cross_model

from utils import dot_similarity, get_pretrained_embedding


In [0]:
DATA_PATH = 'data/all'
PRETRAINED_EMBEDDING_PATH = args.pretrained_embeddings_path
NUM_WORDS = 1000
MAX_LEN = 20
BATCH_SIZE = 32
EMBEDDING_DIM = 100

In [0]:
print('Loading data')
train_gen, val_gen, word_index = get_data(DATA_PATH, NUM_WORDS, MAX_LEN, BATCH_SIZE,
                                          train_test_split=0.8)

matrix_similarity_function = dot_similarity
embedding_matrix = None

if PRETRAINED_EMBEDDING_PATH != "":
    embedding_matrix = get_pretrained_embedding(PRETRAINED_EMBEDDING_PATH, NUM_WORDS + 1,
                                                EMBEDDING_DIM, word_index)

# TODO: create function 'fitting' (useful for HPO)
model = get_deep_cross_model(NUM_WORDS + 1, EMBEDDING_DIM, MAX_LEN, matrix_similarity_function,
                             embedding_matrix)
model.summary()
model.compile('adam', loss='binary_crossentropy', metrics=['accuracy'])
print("Start training")
model.fit(train_gen, validation_data=val_gen, epochs=10, verbose=2)
